# C2
# Предсказание отмены такси

## курс "Машинное обучение 2", программа OzonMasters, 2022

## Студент: Арешин Станислав Олегович

## Baseline 2. Решение на всех данных

В данном ноутбуке представлено решение, побившее второй бейслайн, которое подробно описано в ноутбуке baseline2_solution.ipynb, но с обучением на всех исходных тестовых данных для получения наибольшего скора. Скор решения на паблике 0.74958 при бейслайне 0.69283.

**Подробное описание решение см. baseline2_solution.ipynb. Данный ноутбук предназначен для восстановления самбита. Это не итоговое решение. В итоговом решении я применил блендинг на всех данных с теми же признаками и немного увеличил скор. Ноутбук с блендингом тоже прилагается.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

%matplotlib inline

# Загрузка данных

In [2]:
df_train = pd.read_csv('1_data/train_data_w_dists.csv')
target = pd.read_csv('1_data/train_target.csv')
df_train['target'] = np.array(target)
df_train = df_train.sort_values('due')
print(df_train.shape)
df_train.head()

(1187461, 14)


,dist,due,f_class,lat,lon,s_class,t_class,dist_to_MOSCOW,dist_to_SPB,dist_to_KAZAN,dist_to_N_NOVGOROD,dist_to_VORONEZH,min_dist_to_city,target
823150,17990.125431,2014-01-01 00:09:32.000,econom,55.750130,37.823242,NaN,NaN,0.206023,8.577844,11.243127,6.209581,4.315741,0.206023,1
685195,2322.180999,2014-01-01 00:10:00.000,econom,55.711488,37.884183,NaN,NaN,0.270543,8.649893,11.182532,6.152650,4.259919,0.270543,0
996389,14608.577392,2014-01-01 00:10:00.000,econom,55.633404,37.797595,NaN,NaN,0.217931,8.613152,11.270209,6.246994,4.213747,0.217931,0
497811,20055.569096,2014-01-01 00:10:00.000,econom,55.780380,37.645240,NaN,NaN,0.037198,8.407917,11.420952,6.384175,4.404071,0.037198,0
812346,20585.444093,2014-01-01 00:10:00.000,econom,55.675676,37.500387,NaN,NaN,0.141746,8.335291,11.566730,6.538104,4.360663,0.141746,1


In [3]:
df_test = pd.read_csv('1_data/test_data_w_dists.csv')
df_test = df_test.sort_values('due')
print(df_test.shape)

df_test.head()

(510937, 13)


,dist,due,f_class,lat,lon,s_class,t_class,dist_to_MOSCOW,dist_to_SPB,dist_to_KAZAN,dist_to_N_NOVGOROD,dist_to_VORONEZH,min_dist_to_city
418832,12228.949942,2014-03-06 08:00:00.000,econom,55.709632,37.767186,business,NaN,0.156845,8.548894,11.299542,6.269254,4.295751,0.156845
330723,11792.691777,2014-03-06 08:00:00.000,business,55.867147,37.665435,NaN,NaN,0.121284,8.383043,11.400706,6.357193,4.478395,0.121284
440585,1424.111651,2014-03-06 08:00:00.000,econom,55.747110,37.642815,business,NaN,0.026964,8.422299,11.423571,6.389524,4.373822,0.026964
462500,487.808926,2014-03-06 08:00:00.000,econom,55.744936,37.615057,business,NaN,0.011117,8.399310,11.451344,6.417364,4.381755,0.011117
75315,2503.594134,2014-03-06 08:00:00.000,econom,55.762961,37.619647,business,NaN,0.007513,8.394317,11.446634,6.411183,4.396913,0.007513


# get_dists_from_cities

In [4]:
from scipy.spatial import distance

def get_dists_from_cities(row):
    MOSCOW = np.array([55.755825, 37.617298])
    SPB = np.array([59.934280, 30.335098])
    KAZAN = np.array([55.830433, 49.066082])
    N_NOVGOROD = np.array([56.326790, 44.005989])
    VORONEZH = np.array([51.659908, 39.200089])
    dist_to_MOSCOW = distance.euclidean(row[['lat', 'lon']], MOSCOW)
    dist_to_SPB = distance.euclidean(row[['lat', 'lon']], SPB)
    dist_to_KAZAN = distance.euclidean(row[['lat', 'lon']], KAZAN)
    dist_to_N_NOVGOROD = distance.euclidean(row[['lat', 'lon']], N_NOVGOROD)
    dist_to_VORONEZH = distance.euclidean(row[['lat', 'lon']], VORONEZH)
    min_dist_to_city = np.min([dist_to_MOSCOW, dist_to_SPB, dist_to_KAZAN, dist_to_N_NOVGOROD, dist_to_VORONEZH]) 
    return dist_to_MOSCOW, dist_to_SPB, dist_to_KAZAN, dist_to_N_NOVGOROD, dist_to_VORONEZH, min_dist_to_city

In [5]:
%%time

df[['dist_to_MOSCOW', 'dist_to_SPB', 'dist_to_KAZAN', 'dist_to_N_NOVGOROD',\
          'dist_to_VORONEZH', 'min_dist_to_city']] = df[['lat', 'lon']].apply(\
                                                        get_dists_from_cities, result_type='expand', axis=1)

In [6]:
%%time

df_test[['dist_to_MOSCOW', 'dist_to_SPB', 'dist_to_KAZAN', 'dist_to_N_NOVGOROD',\
          'dist_to_VORONEZH', 'min_dist_to_city']] = df_test[['lat', 'lon']].apply(\
                                                        get_dists_from_cities, result_type='expand', axis=1)

In [7]:
# df.to_csv('1_data/train_data_w_dists.csv', index=False)
# df_test.to_csv('1_data/test_data_w_dists.csv', index=False)

# get_datetime_features

In [10]:
def get_datetime_features(df):
    df_res = df.copy()
    df_res['due_datetime'] = pd.to_datetime(df_res['due'])
    df_res['month'] = df_res['due_datetime'].dt.month
    df_res['day'] = df_res['due_datetime'].dt.day
    df_res['weekday'] = df_res['due_datetime'].dt.weekday
    df_res['is_weekend'] = df_res.apply(lambda row: 1 if  row['weekday'] > 4\
                                           else 0, axis=1)
    df_res['hour'] = df_res['due_datetime'].dt.hour
    df_res['day_period'] = (df_res['hour'] % 24 + 4) // 4
    
    df_res['time_as_str'] = df_res['due'].apply(lambda x: x[x.find(' ') + 1:-4])
    df_res['due_timedelta'] = pd.to_timedelta(df_res['time_as_str'])
    df_res['total_seconds'] = df_res['due_timedelta'].dt.total_seconds()
    df_res['total_minutes'] = df_res['total_seconds'] // 60
    
    df_res.drop(['time_as_str', 'due_timedelta', 'due_datetime', 'due'], inplace=True, axis=1)
    return df_res  

In [11]:
%%time 

df_train = get_datetime_features(df_train)

df_test = get_datetime_features(df_test)

Wall time: 23.6 s


# get_geo_features

В тесте есть перепутанные lat lon

In [12]:
def get_geo_features(df, test=False):
    df_res = df.copy()
    df_res['city'] = 'other'
    df_res.loc[df_res['lat'].between(59.4, 60.3) & df_res['lon'].between(29.23, 31.394), 'city'] = 'spb'
    df_res.loc[df_res['lat'].between(54.35, 56.7) & df_res['lon'].between(35.05, 39.86), 'city'] = 'moscow'
    df_res.loc[df_res['lat'].between(49.68, 52.03) & df_res['lon'].between(37.68, 43.35), 'city'] = 'voronezh'
    df_res.loc[df_res['lat'].between(55.95, 56.45) & df_res['lon'].between(43.38, 44.76), 'city'] = 'nnovgorod'
    df_res.loc[df_res['lat'].between(55.46, 55.94) & df_res['lon'].between(48.54, 50.06), 'city'] = 'kazan'
    
    if test:
        df_res['lat'], df_res['lon'] = np.where(df_res['city']=='other',\
                                                (df_res['lon'],df_res['lat']),(df_res['lat'],df_res['lon']))
        df_res.loc[df_res['lat'].between(59.4, 60.3) & df_res['lon'].between(29.23, 31.394), 'city'] = 'spb'
        df_res.loc[df_res['lat'].between(54.35, 56.7) & df_res['lon'].between(35.05, 39.86), 'city'] = 'moscow'
        df_res.loc[df_res['lat'].between(49.68, 52.03) & df_res['lon'].between(37.68, 43.35), 'city'] = 'voronezh'
        df_res.loc[df_res['lat'].between(55.95, 56.45) & df_res['lon'].between(43.38, 44.76), 'city'] = 'nnovgorod'
        df_res.loc[df_res['lat'].between(55.46, 55.94) & df_res['lon'].between(48.54, 50.06), 'city'] = 'kazan'
    return df_res  

In [13]:
%%time

df_train = get_geo_features(df_train)
df_test = get_geo_features(df_test, test=True)

Wall time: 268 ms


In [14]:
df_train.columns

Index(['dist', 'f_class', 'lat', 'lon', 's_class', 't_class', 'dist_to_MOSCOW',
       'dist_to_SPB', 'dist_to_KAZAN', 'dist_to_N_NOVGOROD',
       'dist_to_VORONEZH', 'min_dist_to_city', 'target', 'month', 'day',
       'weekday', 'is_weekend', 'hour', 'day_period', 'total_seconds',
       'total_minutes', 'city'],
      dtype='object')

# find_almost_duplicates

In [15]:
# предварительно отсортировать данные
def find_almost_dupclicates(df):
    df_res = df.copy()
    df_res['lat_lon_duplicated'] = df_res.duplicated(subset=['lat', 'lon'], keep='last')
    df_res['dist_lat_lon_duplicated'] = df_res.duplicated(subset=['dist', 'lat', 'lon'], keep='last')
    df_res['dist_lat_lon_classes_duplicated'] = df_res.duplicated(subset=['dist', 'lat', 'lon', 'f_class','s_class','t_class'], keep='last')
    df_res['day_lat_lon_duplicated'] = df_res.duplicated(subset=['month', 'day', 'lat', 'lon'], keep='last')
    df_res['dayperiod_lat_lon_duplicated'] = df_res.duplicated(subset=['month', 'day','day_period', 'lat', 'lon'], keep='last')
    df_res['hour_lat_lon_duplicated'] = df_res.duplicated(subset=['month', 'day','hour', 'lat', 'lon'], keep='last')
    df_res['minutes_lat_lon_duplicated'] = df_res.duplicated(subset=['month', 'day','total_minutes', 'lat', 'lon'], keep='last')
    df_res['dist_day_lat_lon_duplicated'] = df_res.duplicated(subset=['dist','month', 'day', 'lat', 'lon'], keep='last')
    df_res['dist_dayperiod_lat_lon_duplicated'] = df_res.duplicated(subset=['dist','month', 'day','day_period', 'lat', 'lon'], keep='last')
    df_res['dist_hour_lat_lon_duplicated'] = df_res.duplicated(subset=['dist','month', 'day','hour', 'lat', 'lon'], keep='last')
    return df_res

In [16]:
df_train = find_almost_dupclicates(df_train)
df_test = find_almost_dupclicates(df_test)

# Шафлим train, восстанавливем порядок test

In [17]:
df_train = df_train.sample(frac=1, random_state=42)
df_test = df_test.sort_index()
df_test.head()

,dist,f_class,lat,lon,s_class,t_class,dist_to_MOSCOW,dist_to_SPB,dist_to_KAZAN,dist_to_N_NOVGOROD,...,lat_lon_duplicated,dist_lat_lon_duplicated,dist_lat_lon_classes_duplicated,day_lat_lon_duplicated,dayperiod_lat_lon_duplicated,hour_lat_lon_duplicated,minutes_lat_lon_duplicated,dist_day_lat_lon_duplicated,dist_dayperiod_lat_lon_duplicated,dist_hour_lat_lon_duplicated
0,4611.506701,econom,55.776618,37.682503,NaN,NaN,0.068440,8.442187,11.383706,6.347375,...,False,False,False,False,False,False,False,False,False,False
1,14450.067441,econom,55.754659,37.810072,NaN,NaN,0.192778,8.564138,11.256265,6.222276,...,True,False,False,False,False,False,False,False,False,False
2,17755.993723,business,55.726141,37.587407,NaN,NaN,0.042127,8.384773,11.479149,6.446626,...,False,False,False,False,False,False,False,False,False,False
3,8533.172059,econom,55.794178,37.576456,business,NaN,0.056027,8.341325,11.489683,6.451556,...,True,False,False,False,False,False,False,False,False,False
4,-1.000000,econom,55.757409,37.542186,business,NaN,0.075129,8.329968,11.524127,6.488832,...,True,True,False,False,False,False,False,False,False,False


# Избавляемся от нанов

In [18]:
cols_to_fill_newcat1 = ['f_class', 's_class', 't_class']

In [19]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='constant', fill_value='unknown')

to_fill_train = np.array(df_train[cols_to_fill_newcat1])
fill_result_train = imp.fit_transform(to_fill_train)
df_train[cols_to_fill_newcat1] = fill_result_train

to_fill_test = np.array(df_test[cols_to_fill_newcat1])
fill_result_test = imp.transform(to_fill_test)
df_test[cols_to_fill_newcat1] = fill_result_test

In [20]:
def is_feature(df, val_cols, cat_cols):
    df_res = df.copy()
    for col in val_cols:
        df_res[f'is_{col}'] = (df_res[col] != -1.0).astype(int)
    for col in cat_cols:
        df_res[f'is_{col}'] = (df_res[col] != 'unknown').astype(int)
    return df_res

In [21]:
val_cols =  ['dist']
cat_cols = ['f_class', 's_class', 't_class']
df_train = is_feature(df_train, val_cols, cat_cols)
df_test = is_feature(df_test, val_cols, cat_cols)

# Encoding

In [22]:
df_train.drop(['city'], axis=1, inplace=True)
df_test.drop(['city'], axis=1, inplace=True)

In [23]:
df_train.columns

Index(['dist', 'f_class', 'lat', 'lon', 's_class', 't_class', 'dist_to_MOSCOW',
       'dist_to_SPB', 'dist_to_KAZAN', 'dist_to_N_NOVGOROD',
       'dist_to_VORONEZH', 'min_dist_to_city', 'target', 'month', 'day',
       'weekday', 'is_weekend', 'hour', 'day_period', 'total_seconds',
       'total_minutes', 'lat_lon_duplicated', 'dist_lat_lon_duplicated',
       'dist_lat_lon_classes_duplicated', 'day_lat_lon_duplicated',
       'dayperiod_lat_lon_duplicated', 'hour_lat_lon_duplicated',
       'minutes_lat_lon_duplicated', 'dist_day_lat_lon_duplicated',
       'dist_dayperiod_lat_lon_duplicated', 'dist_hour_lat_lon_duplicated',
       'is_dist', 'is_f_class', 'is_s_class', 'is_t_class'],
      dtype='object')

In [24]:
y_train = np.array(df_train['target'])

df_train.drop(['target'], axis=1, inplace=True)

In [25]:
from category_encoders import OrdinalEncoder

cat_columns = list(df_train.select_dtypes('object').columns)
encoder = OrdinalEncoder(cols = cat_columns)

df_train_encoded = encoder.fit_transform(df_train)
df_test_encoded = encoder.transform(df_test)

print(df_train_encoded.shape)
print(df_test_encoded.shape)

df_train_encoded.head()

(1187461, 34)
(510937, 34)


,dist,f_class,lat,lon,s_class,t_class,dist_to_MOSCOW,dist_to_SPB,dist_to_KAZAN,dist_to_N_NOVGOROD,...,dayperiod_lat_lon_duplicated,hour_lat_lon_duplicated,minutes_lat_lon_duplicated,dist_day_lat_lon_duplicated,dist_dayperiod_lat_lon_duplicated,dist_hour_lat_lon_duplicated,is_dist,is_f_class,is_s_class,is_t_class
588047,3159.514223,1,55.722563,37.612668,1,1,0.033583,8.408423,11.453922,6.421810,...,False,False,False,False,False,False,1,1,0,0
772639,11202.288951,1,55.673114,37.536547,1,1,0.115593,8.367700,11.530608,6.502382,...,False,False,False,False,False,False,1,1,0,0
399103,2839.340896,1,59.933834,30.355178,2,1,8.378195,0.020085,19.155569,14.119327,...,False,False,False,False,False,False,1,1,1,0
843457,-1.000000,2,55.762153,37.652895,1,1,0.036155,8.423586,11.413391,6.378136,...,False,False,False,False,False,False,0,1,0,0
810593,4790.515817,1,55.754203,37.556388,2,1,0.060932,8.343864,11.509946,6.474968,...,True,True,False,False,False,False,1,1,1,0


In [27]:
df_train_encoded.columns

Index(['dist', 'f_class', 'lat', 'lon', 's_class', 't_class', 'dist_to_MOSCOW',
       'dist_to_SPB', 'dist_to_KAZAN', 'dist_to_N_NOVGOROD',
       'dist_to_VORONEZH', 'min_dist_to_city', 'month', 'day', 'weekday',
       'is_weekend', 'hour', 'day_period', 'total_seconds', 'total_minutes',
       'lat_lon_duplicated', 'dist_lat_lon_duplicated',
       'dist_lat_lon_classes_duplicated', 'day_lat_lon_duplicated',
       'dayperiod_lat_lon_duplicated', 'hour_lat_lon_duplicated',
       'minutes_lat_lon_duplicated', 'dist_day_lat_lon_duplicated',
       'dist_dayperiod_lat_lon_duplicated', 'dist_hour_lat_lon_duplicated',
       'is_dist', 'is_f_class', 'is_s_class', 'is_t_class'],
      dtype='object')

# Creating model

In [28]:
X_train = np.array(df_train_encoded)
X_test = np.array(df_test_encoded)


print(f'Train shape : {X_train.shape}')
print(f'Test shape : {X_test.shape}')

Train shape : (1187461, 34)
Test shape : (510937, 34)


## Catboost

In [29]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=1000,
                          loss_function = 'Logloss',
                          task_type="CPU",
                           max_depth=9,
                            learning_rate= 0.1,
                           random_state=1
                          )

model.fit(X = X_train,
          y = y_train, 
          verbose = 100,
          early_stopping_rounds=50)

0:	learn: 0.6160100	total: 475ms	remaining: 7m 54s
100:	learn: 0.3385846	total: 30.7s	remaining: 4m 32s
200:	learn: 0.3344264	total: 1m 1s	remaining: 4m 6s
300:	learn: 0.3311566	total: 1m 33s	remaining: 3m 38s
400:	learn: 0.3285554	total: 2m 6s	remaining: 3m 8s
500:	learn: 0.3261217	total: 2m 38s	remaining: 2m 37s
600:	learn: 0.3238266	total: 3m 10s	remaining: 2m 6s
700:	learn: 0.3216506	total: 3m 43s	remaining: 1m 35s
800:	learn: 0.3195623	total: 4m 16s	remaining: 1m 3s
900:	learn: 0.3175525	total: 4m 51s	remaining: 32s
999:	learn: 0.3156444	total: 5m 27s	remaining: 0us


In [30]:
y_pred = model.predict_proba(X_test)[:, 1]

## Result

In [31]:
submission = pd.read_csv('1_data/sample_submission.csv')
submission.head()

,index,target
0,0,0.590232
1,1,0.693891
2,2,0.374479
3,3,0.319133
4,4,0.316124


In [32]:
submission['target'] = y_pred
submission.to_csv('cb_duplicate_features3.csv', index=False)

In [32]:
submission

,index,target
0,0,0.056141
1,1,0.046842
2,2,0.064439
3,3,0.061540
4,4,0.259427
...,...,...
510932,510932,0.360546
510933,510933,0.192667
510934,510934,0.194705
510935,510935,0.274525
